# Inference from a model deployed on AIcore

https://pypi.org/project/ai-core-sdk/  # pip install ai-core-sdk

pip install ai-api-client-sdk

https://pypi.org/project/generative-ai-hub-sdk/  

In [12]:
# ! pip install ai-core-sdk
# ! pip install ai-api-client-sdk
# ! pip install generative-ai-hub-sdk

In [13]:
def signavio_prompt(description_llm):
    """
    """
    #import yaml

    mistral="""
  <s>[INST]
  You are SIGNAL assistant, a part of SAP Signavio's Process Intelligence Suite. 
  SIGNAL stands for Signavio Analytics Language. 
  SIGNAL is a dialect of SQL.
  Your goal is to help users craft SIGNAL 
  queries and understand the SIGNAL language better. 
  
  
  ### Instruction:
  
  Task to solve:
  
  Construct SIGNAL expression querying given textual description in Input.
  Expected output results are in JSON format.
  
  
  ### Input:
  {description_llm}
  
  ### Output:
  [/INST]
"""
    #mistral_json=yaml.load(mistral,  Loader=yaml.FullLoader)
    #return mistral_json["query_template"].format(description_llm=str(description_llm))
    return mistral.format(description_llm=description_llm) 

description_llm="Calculate the sum of the 'Inv. Converted USD Amount' column in the 'defaultview-225' table, filtered by the conditions: 'Last Inv. Item Cash Discount Days 1' is less than 'Last Inv. Item Vendor Master CD 1', or 'Last Inv. Item Cash Discount Days 2' is less than 'Last Inv. Item Vendor Master CD 2', or 'Last Inv. Item Net Payment Terms Period' is less than 'Last Inv. Item Vendor Master NDD'."

print(signavio_prompt(description_llm))


  <s>[INST]
  You are SIGNAL assistant, a part of SAP Signavio's Process Intelligence Suite. 
  SIGNAL stands for Signavio Analytics Language. 
  SIGNAL is a dialect of SQL.
  Your goal is to help users craft SIGNAL 
  queries and understand the SIGNAL language better. 
  
  
  ### Instruction:
  
  Task to solve:
  
  Construct SIGNAL expression querying given textual description in Input.
  Expected output results are in JSON format.
  
  
  ### Input:
  Calculate the sum of the 'Inv. Converted USD Amount' column in the 'defaultview-225' table, filtered by the conditions: 'Last Inv. Item Cash Discount Days 1' is less than 'Last Inv. Item Vendor Master CD 1', or 'Last Inv. Item Cash Discount Days 2' is less than 'Last Inv. Item Vendor Master CD 2', or 'Last Inv. Item Net Payment Terms Period' is less than 'Last Inv. Item Vendor Master NDD'.
  
  ### Output:
  [/INST]



In [4]:
from gen_ai_hub.proxy import GenAIHubProxyClient
from gen_ai_hub.proxy.native.openai import OpenAI
 
GenAIHubProxyClient.add_foundation_model_scenario(
    scenario_id='signavio-llm',
    config_names='text2signal-*',
    prediction_url_suffix='/v1/completions'
)
 
openai = OpenAI()
print(openai.completions.create(model_name='chatbd', prompt="what is the average cycle time?"))

Completion(id='cmpl-240b5f4fc4a44b87aff01a77b7a97368', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' A cycle time in lean manufacturing is the amount of time a project job requires,')], created=64164, model='chatbd', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=8, total_tokens=24))


In [23]:
description_llm="Calculate the sum of the 'Inv. Converted USD Amount' column in the 'defaultview-225' table, filtered by the conditions: 'Last Inv. Item Cash Discount Days 1' is less than 'Last Inv. Item Vendor Master CD 1', or 'Last Inv. Item Cash Discount Days 2' is less than 'Last Inv. Item Vendor Master CD 2', or 'Last Inv. Item Net Payment Terms Period' is less than 'Last Inv. Item Vendor Master NDD'."

prompt=signavio_prompt(description_llm)
completion = openai.completions.create(model_name='chatbd', 
                                prompt=prompt, 
                                max_tokens=1500,
                                temperature=0
)

In [26]:
import json
#query_llm = json.loads(res.model_dump_json(indent=2))["choices"][0]["text"]

res =json.loads(completion.model_dump_json(indent=2))["usage"]
res_q = json.loads(completion.model_dump_json(indent=2))
finish_reason = res_q['choices'][0]['finish_reason']
query_q=res_q['choices'][0]['text'] #+' """}'

try:
        #processed = '"""'.join(query_q.split('"""', 2)[:2]) + '"""}'
        #q_out=eval(processed.encode())["query"]
        q_out=eval(query_q)["query"]
except Exception as e:
        print(e,"\n>>>>>>>>>>>>>>>Problem to process", query_q)
        q_out=f"Error {e}"

print(finish_reason,query_q)

unterminated string literal (detected at line 6) (<string>, line 6) 
>>>>>>>>>>>>>>>Problem to process { "query":"""SELECT 
SUM("Inv. Converted USD Amount") 
FILTER (WHERE ("Last Inv. Item Cash Discount Days 1") < "Last Inv. Item Vendor Master CD 1" 
OR ("Last Inv. Item Cash Discount Days 2" < "Last Inv. Item Vendor Master CD 2"
OR "Last Inv. Item Net Payment Terms Period" < "Last Inv. Item Vendor Master NDD"))
FROM "defaultview-225""""}</s>

stop { "query":"""SELECT 
SUM("Inv. Converted USD Amount") 
FILTER (WHERE ("Last Inv. Item Cash Discount Days 1") < "Last Inv. Item Vendor Master CD 1" 
OR ("Last Inv. Item Cash Discount Days 2" < "Last Inv. Item Vendor Master CD 2"
OR "Last Inv. Item Net Payment Terms Period" < "Last Inv. Item Vendor Master NDD"))
FROM "defaultview-225""""}</s>



In [6]:
# Read credentials from .env
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import sys
import os
sys.path.append("./")
env_path = Path('..') / '.env_aicore'
load_dotenv(dotenv_path=env_path)

True

In [11]:
os.getenv("CLIENT_SECRET")
os.getenv("resource_group_id")

'default'

In [8]:
# https://pypi.org/project/ai-api-client-sdk/
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client

# Instantiate the client
ai_core_client = AIAPIV2Client(base_url=os.getenv("AI_API_BASE"),
                        auth_url=os.getenv("AUTH_URL"),
                        client_id=os.getenv("CLIENT_ID"),
                        client_secret=os.getenv("CLIENT_SECRET"),
                        resource_group=os.getenv("resource_group_id"))

# Make some queries, e.g.
scenarios = ai_core_client.scenario.query()

AIAPIServerException: Failed to get /scenarios:  
 Status Code: 404, Request ID:None

In [ ]:
DEPLOYMENT_ID = "d3560fac61da06c3"
response = ai_core_client.deployment.get(
    deployment_id=DEPLOYMENT_ID,
    resource_group="default"
)

print(response.__dict__)

In [ ]:
import requests

# URL
deployment_url = "https://api.ai.internalprod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d3560fac61da06c3"

# Preparing the input for inference
# {"message":"Calculate the sum of the 'Inv. Converted USD Amount' column in the 'defaultview-225' table, filtered by the conditions: 'Last Inv. Item Cash Discount Days 1' is less than 'Last Inv. Item Vendor Master CD 1', or 'Last Inv. Item Cash Discount Days 2' is less than 'Last Inv. Item Vendor Master CD 2', or 'Last Inv. Item Net Payment Terms Period' is less than 'Last Inv. Item Vendor Master NDD'",
# "model":"signal-llama","analytics":true,
# "parameters":"{\"max_tokens\":\"1500\",\"temperature\":\"0\"}"}

# curl http://localhost:8000/v1/completions \
#    -H "Content-Type: application/json" \
#    -d '{
#        "model": "our-awesome-model/v1",
#        "prompt": "San Francisco is a",
#        "max_tokens": 7,
#        "temperature": 0
#    }'


test_input = {
    "model": "chatbd", # modelName	
    "prompt": signavio_prompt(description_llm),
    #"analytics":true,
    "parameters":"{\"max_tokens\":\"1500\",\"temperature\":\"0\"}"
    
}

#endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
endpoint = f"{deployment_url}/v1/completions"
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "default",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)

print('Inference result:', response.json())

In [ ]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

ai_core_client = AICoreV2Client(base_url=os.getenv("AI_API_BASE"),
                        auth_url=os.getenv("AUTH_URL"),
                        client_id=os.getenv("CLIENT_ID"),
                        client_secret=os.getenv("CLIENT_SECRET"),
                        resource_group=os.getenv("resource_group_id"))

In [ ]:
response = ai_core_client.deployment.get(
    deployment_id="YOUR_DEPLOYMENT_ID",
    resource_group="default"
)

print(response.__dict__)

# Create widget from Signavio API

In [29]:
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import sys
import os
sys.path.append("./")
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

! echo $MY_SIGNAVIO_NAME

alexey.streltsov@sap.com


In [32]:
# Class coped and modified  from https://github.com/signavio/bpmn2constraints/blob/main/tutorial/SignavioAuthenticator.py
class SignavioAuthenticatorCoE:
    def __init__(self, system_instance, tenant_id, email, pw):
        self.system_instance = system_instance
        self.tenant_id = tenant_id
        self.email = email
        self.pw = pw

    """
    Takes care of authentication against Signavio systems
    """

    def authenticate(self):
        """
        Authenticates user at Signavio system instance and initiates session.
        Returns:
            dictionary: Session information
        """
        login_url = self.system_instance + "/p/login"
        data = {"name": self.email, "password": self.pw, "tokenonly": "true"}
        #if "tenant_id" in locals():
        #    data["tenant"] = self.tenant_id
        data["tenant"] = self.tenant_id
        # authenticate
        login_request = requests.post(login_url, data)

        # retrieve token and session ID
        auth_token = login_request.content.decode("utf-8")
        jsesssion_ID = login_request.cookies["JSESSIONID"]

        # The cookie is named 'LBROUTEID' for base_url 'editor.signavio.com'
        # and 'editor.signavio.com', and 'AWSELB' for base_url
        # 'app-au.signavio.com' and 'app-us.signavio.com'
        lb_route_ID = login_request.cookies["LBROUTEID"]

        # return credentials
        return {
            "jsesssion_ID": jsesssion_ID,
            "lb_route_ID": lb_route_ID,
            "auth_token": auth_token,
        }

def credentials_actualization(system_instance, workspace_id, user_name, pw, auth={}):
    # get credentials for current session. They are valid for 24 hours 
    authenticator = SignavioAuthenticatorCoE(system_instance, workspace_id, user_name, pw)
    auth_data = authenticator.authenticate()
    cookies = {'JSESSIONID': auth_data['jsesssion_ID'], 'LBROUTEID': auth_data['lb_route_ID']}
    headers = {'Accept': 'application/json', 'x-signavio-id':  auth_data['auth_token']}
    print(auth_data['jsesssion_ID'],auth_data['lb_route_ID'])
    print(auth_data['auth_token'])
    auth[workspace_name]={"cookies":cookies,"headers":headers}
    return auth    

# POST - need to re-run to inject the above generated credentials
#q=q_list_processes #

#def POST_Signavio(query={},cookies=cookies,headers=headers):
def POST_Signavio(query={},workspace_name="",auth=auth):
        signal_endpoint = system_instance + '/g/api/pi-graphql/graphql'
        timeout=50
        try:
            query_request = requests.post(
                signal_endpoint,
                cookies=auth[workspace_name]["cookies"],
                headers=auth[workspace_name]["headers"],
                timeout=timeout,
                json=query)
            query_request.raise_for_status()
        except requests.HTTPError as ex:
            # possibly check response for a message
            raise ex  # let the caller handle it
        except requests.Timeout:
            print("request took too long")


        return query_request.json()

In [31]:
# select workspace here 
import json
import pathlib
import logging
import requests
import os
logging.getLogger("urllib3").setLevel(logging.WARNING)


auth={}
 
user_name = os.environ.get('MY_SIGNAVIO_NAME','alexey.streltsov@sap.com') # username
pw = os.environ.get('MY_SIGNAVIO_PASSWORD', None) # Signavio password
print("Your are:",user_name)
system_instance = 'https://editor.signavio.com'
workspace_id = 'b0f07deabd3140aea5344baa686e0d84' # workspace Process AI 
workspace_name="Process AI"
credentials_actualization(system_instance, workspace_id, user_name, pw, auth=auth) 
#workspace_id = 'ccf98b692d1d4e0aa895df3aab8cd905' # workspace Solutions Demo Workspace access by alexandra.dang@sap.com
#workspace_name="Solutions Demo Workspace"
#credentials_actualization(system_instance, workspace_id, user_name, pw, auth=auth)
# This workspace is under discussion with sarah.roessler@sap.com
#workspace_id = "2da9ebfecca0479398ad9905b88c5187"
#workspace_name="SAP Signavio Suite Power Challenge"
#user_name="demo+113@signavio.com"
#pw="signavio"
#credentials_actualization(system_instance, workspace_id, user_name, pw, auth=auth)

auth

Your are: alexey.streltsov@sap.com
F32C16BEEBEDDDD815F0270506AD2D08 "4e9835474540c554"
8719156911334fb78be943b322144a09


{'Process AI': {'cookies': {'JSESSIONID': 'F32C16BEEBEDDDD815F0270506AD2D08',
   'LBROUTEID': '"4e9835474540c554"'},
  'headers': {'Accept': 'application/json',
   'x-signavio-id': '8719156911334fb78be943b322144a09'}}}

In [33]:
q_list_processes={
    "operationName": "Subjects",
    "variables": {},
    "query": "query Subjects {\n  subjects {\n    id\n    name\n    businessArea\n  investigationCount dashboardCount }}"
}

workspace_name="Process AI"
list_processes_1 =  POST_Signavio(query=q_list_processes,workspace_name=workspace_name)['data']['subjects']
print(f"In Workspace Name: {workspace_name}. Number of processes: {len(list_processes_1)}")

In Workspace Name: Process AI. Number of processes: 6


In [36]:
#query_to_api.find("null")

171

In [37]:
#def create_widget(name="rr",):
query_to_api= {
    "operationName": "CreateWidgetOnInvestigation",
    "variables": {
        "id": "qq-1",
        "widget": {
            "visualization": {
                "unit":  None,
                "type": "VALUE"
            },
            "dataSource": {
                "query": "SELECT count(case_id) FROM THIS_PROCESS",
                "type": "SIGNAL"
            },
            "name": "rr1"
        }
    },
    "query": "mutation CreateWidgetOnInvestigation($id: ID!, $widget: JSON!) {\n  createWidget(id: $id, widget: $widget) {\n    ...Widget\n    __typename\n  }\n}\n\nfragment Widget on Widget {\n  id\n  name\n  description\n  children\n  filters {\n    ...Filter\n    __typename\n  }\n  visualization {\n    ...Visualization\n    __typename\n  }\n  dataSource {\n    ...WidgetDataSource\n    __typename\n  }\n  __typename\n}\n\nfragment Filter on Filter {\n  ... on ActivityOccurrencesFilter {\n    ...ActivityOccurrencesFilter\n    __typename\n  }\n  ... on CustomSignalFilter {\n    ...CustomSignalFilter\n    __typename\n  }\n  ... on BehaviorFilter {\n    ...BehaviorFilter\n    __typename\n  }\n  ... on CaseCycleTimeFilter {\n    ...CaseCycleTimeFilter\n    __typename\n  }\n  ... on CaseDateFilter {\n    ...CaseDateFilter\n    __typename\n  }\n  ... on EventDateFilter {\n    ...EventDateFilter\n    __typename\n  }\n  ... on CaseDateRollingFilter {\n    ...CaseDateRollingFilter\n    __typename\n  }\n  ... on CaseEventCountFilter {\n    ...CaseEventCountFilter\n    __typename\n  }\n  ... on CaseIdFilter {\n    ...CaseIdFilter\n    __typename\n  }\n  ... on ProcessDiscoveryFilter {\n    ...ProcessDiscoveryFilter\n    __typename\n  }\n  ... on ProcessModelFilter {\n    ...ProcessModelFilter\n    __typename\n  }\n  ... on ProcessVarChoiceFilter {\n    ...ProcessVarChoiceFilter\n    __typename\n  }\n  ... on ProcessVarCurrencyFilter {\n    ...ProcessVarCurrencyFilter\n    __typename\n  }\n  ... on ProcessVarDateTimeFilter {\n    ...ProcessVarDateTimeFilter\n    __typename\n  }\n  ... on ProcessVarDoublePrecisionFilter {\n    ...ProcessVarDoublePrecisionFilter\n    __typename\n  }\n  ... on ProcessVarDurationFilter {\n    ...ProcessVarDurationFilter\n    __typename\n  }\n  __typename\n}\n\nfragment ActivityOccurrencesFilter on ActivityOccurrencesFilter {\n  id\n  type\n  includes\n  excludes\n  enabled\n  level\n  __typename\n}\n\nfragment CustomSignalFilter on CustomSignalFilter {\n  id\n  type\n  name\n  description\n  signalFragment\n  enabled\n  level\n  __typename\n}\n\nfragment BehaviorFilter on BehaviorFilter {\n  id\n  type\n  behaviorType\n  startEndType\n  startEndEvents\n  sequenceType\n  sequenceSource\n  sequenceTarget\n  reworkType\n  reworkOperator\n  reworkFrequency\n  enabled\n  __typename\n}\n\nfragment CaseCycleTimeFilter on CaseCycleTimeFilter {\n  id\n  type\n  maximum {\n    value\n    unit\n    operator\n    __typename\n  }\n  minimum {\n    value\n    unit\n    operator\n    __typename\n  }\n  enabled\n  __typename\n}\n\nfragment CaseDateFilter on CaseDateFilter {\n  type\n  startDate\n  endDate\n  id\n  caseInclusionCriterion\n  enabled\n  __typename\n}\n\nfragment EventDateFilter on EventDateFilter {\n  id\n  type\n  startDate\n  endDate\n  enabled\n  __typename\n}\n\nfragment CaseDateRollingFilter on CaseDateRollingFilter {\n  id\n  type\n  unit\n  amount\n  caseInclusionCriterion\n  enabled\n  __typename\n}\n\nfragment CaseEventCountFilter on CaseEventCountFilter {\n  id\n  type\n  minimum {\n    value\n    operator\n    __typename\n  }\n  maximum {\n    value\n    operator\n    __typename\n  }\n  enabled\n  __typename\n}\n\nfragment CaseIdFilter on CaseIdFilter {\n  id\n  type\n  includes\n  excludes\n  enabled\n  __typename\n}\n\nfragment ProcessDiscoveryFilter on ProcessDiscoveryFilter {\n  id\n  type\n  dataSource\n  variantsAsEventNames\n  aggregationDimension\n  order\n  enabled\n  __typename\n}\n\nfragment ProcessModelFilter on ProcessModelFilter {\n  id\n  type\n  dataSource\n  variantsAsEventNames\n  aggregationDimension\n  order\n  enabled\n  __typename\n}\n\nfragment ProcessVarChoiceFilter on ProcessVarChoiceFilter {\n  id\n  type\n  processVariable\n  includes\n  excludes\n  enabled\n  parameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    __typename\n  }\n  level\n  __typename\n}\n\nfragment ProcessVarCurrencyFilter on ProcessVarCurrencyFilter {\n  id\n  type\n  processVariable\n  minimum {\n    value\n    operator\n    __typename\n  }\n  maximum {\n    value\n    operator\n    __typename\n  }\n  enabled\n  parameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    __typename\n  }\n  level\n  __typename\n}\n\nfragment ProcessVarDateTimeFilter on ProcessVarDateTimeFilter {\n  startDate\n  type\n  endDate\n  id\n  processVariable\n  enabled\n  parameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    __typename\n  }\n  level\n  __typename\n}\n\nfragment ProcessVarDoublePrecisionFilter on ProcessVarDoublePrecisionFilter {\n  id\n  type\n  processVariable\n  minimum {\n    value\n    operator\n    __typename\n  }\n  maximum {\n    value\n    operator\n    __typename\n  }\n  enabled\n  parameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    __typename\n  }\n  level\n  __typename\n}\n\nfragment ProcessVarDurationFilter on ProcessVarDurationFilter {\n  id\n  type\n  processVariable\n  minimum {\n    value\n    unit\n    operator\n    __typename\n  }\n  maximum {\n    value\n    unit\n    operator\n    __typename\n  }\n  enabled\n  parameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    __typename\n  }\n  level\n  __typename\n}\n\nfragment Visualization on Visualization {\n  ... on ActivityList {\n    type\n    __typename\n  }\n  ... on Area {\n    type\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on BarChart {\n    type\n    orientation\n    grouping\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on CasesTable {\n    type\n    columnConfigurations {\n      ...ColumnConfiguration\n      __typename\n    }\n    __typename\n  }\n  ... on Chapter {\n    type\n    __typename\n  }\n  ... on FuzzyModel {\n    type\n    edgeGranularity\n    nodeGranularity\n    annotateBy\n    width\n    __typename\n  }\n  ... on DualAxisChart {\n    type\n    orientation\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on HeatMap {\n    type\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on Histogram {\n    type\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    unit {\n      ...Unit\n      __typename\n    }\n    isAverageShown\n    percentilesShown\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on PieChart {\n    type\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on ProcessDiagram {\n    type\n    modelId\n    revisionId\n    __typename\n  }\n  ... on ProcessDiscovery {\n    type\n    graphRenderingMode\n    edgeDimension\n    variantSelection {\n      ...VariantSelection\n      __typename\n    }\n    __typename\n  }\n  ... on ProcessConformance {\n    type\n    selectedView\n    variantSelection {\n      ...VariantSelection\n      __typename\n    }\n    __typename\n  }\n  ... on OlapTable {\n    type\n    columnConfigurations {\n      ...ColumnConfiguration\n      __typename\n    }\n    __typename\n  }\n  ... on SankeyChart {\n    type\n    width\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on Spreadsheet {\n    type\n    __typename\n  }\n  ... on Table {\n    type\n    grouped\n    columnConfigurations {\n      ...ColumnConfiguration\n      __typename\n    }\n    __typename\n  }\n  ... on Text {\n    type\n    __typename\n  }\n  ... on Throughput {\n    type\n    __typename\n  }\n  ... on TimeSeries {\n    type\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    unit {\n      ...Unit\n      __typename\n    }\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on TreeMap {\n    type\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  ... on Value {\n    type\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    unit {\n      ...Unit\n      __typename\n    }\n    rounding {\n      ...Rounding\n      __typename\n    }\n    __typename\n  }\n  ... on ScatterPlot {\n    type\n    hasRegressionLine\n    dataLabel {\n      ...DataLabel\n      __typename\n    }\n    colorPalette {\n      ...ColorPalette\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment Threshold on Threshold {\n  type\n  visible\n  baseColor\n  goal\n  values {\n    label\n    colorAbove\n    value\n    __typename\n  }\n  __typename\n}\n\nfragment Unit on WidgetUnit {\n  ... on SuffixUnit {\n    type\n    suffix\n    __typename\n  }\n  ... on CurrencyUnit {\n    type\n    currencyCode\n    __typename\n  }\n  ... on DurationUnit {\n    type\n    __typename\n  }\n  __typename\n}\n\nfragment Rounding on NumberRoundingSettings {\n  mode\n  numberOfDecimals\n  __typename\n}\n\nfragment ColumnConfiguration on ColumnConfiguration {\n  columnName\n  columnAlias\n  columnPosition\n  settings {\n    thresholds {\n      ...Threshold\n      __typename\n    }\n    dateTimeFormat {\n      ... on SignalWidgetDateFormattingDateAndTime {\n        year\n        month\n        day\n        hour\n        minute\n        __typename\n      }\n      ... on SignalWidgetDateFormattingDate {\n        year\n        month\n        day\n        __typename\n      }\n      ... on SignalWidgetDateFormattingTime {\n        hour\n        minute\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment VariantSelection on VariantSelection {\n  dataSource\n  aggregationDimension\n  order\n  selectedVariants\n  sortBy\n  aggregationDimensionParameter {\n    ... on ProcessVariableParameter {\n      type\n      name\n      __typename\n    }\n    ... on CustomAttributeParameter {\n      type\n      customAttributeId\n      __typename\n    }\n    ... on MetricParameter {\n      type\n      metricId\n      __typename\n    }\n    ... on SemanticTypeParameter {\n      type\n      semanticType\n      __typename\n    }\n    __typename\n  }\n  limit\n  offset\n  __typename\n}\n\nfragment DataLabel on DataLabel {\n  show\n  rotate\n  fontSize\n  fontWeight\n  fontStyle\n  position\n  __typename\n}\n\nfragment ColorPalette on ColorPalette {\n  id\n  reversed\n  __typename\n}\n\nfragment WidgetDataSource on WidgetDataSource {\n  ... on StructuredWidgetDataSource {\n    type\n    projections {\n      ... on ParameterizedProjection {\n        type\n        attribute\n        aggregation\n        parameter {\n          ... on ProcessVariableParameter {\n            type\n            name\n            __typename\n          }\n          ... on CustomAttributeParameter {\n            type\n            customAttributeId\n            __typename\n          }\n          ... on SemanticTypeParameter {\n            type\n            semanticType\n            __typename\n          }\n          __typename\n        }\n        order\n        __typename\n      }\n      ... on MetricProjection {\n        type\n        metric\n        order\n        __typename\n      }\n      __typename\n    }\n    groupBy {\n      ... on AttributeGroupBy {\n        type\n        attribute\n        parameter {\n          ... on ProcessVariableParameter {\n            type\n            name\n            __typename\n          }\n          ... on CustomAttributeParameter {\n            type\n            customAttributeId\n            __typename\n          }\n          ... on SemanticTypeParameter {\n            type\n            semanticType\n            __typename\n          }\n          __typename\n        }\n        order\n        __typename\n      }\n      __typename\n    }\n    timeBucket\n    limit {\n      ... on Limit {\n        value\n        enabled\n        showWarning\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  ... on SignalWidgetDataSource {\n    type\n    query\n    __typename\n  }\n  ... on StaticWidgetDataSource {\n    type\n    data\n    __typename\n  }\n  __typename\n}\n"
}
